# RoBERTa-large

## Dependencies

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install -U nlp

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-5glumcib
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-5glumcib
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.1MB 3.4MB/s 
     |████████████████████████████████| 3.0MB 8.7MB/s 
     |████████████████████████████████| 890kB 39.6MB/s 
  Created wheel for transformers: filename=transformers-3.2.0-cp36-none-any.whl size=1058986 sha256=8a5b2ee75ef174aafd223ff30f24b136a31c5be9ef7fa8957bbf1d829e07ea18
  Stored in directory: /tmp/pip-ephem-wheel-cache-rqa9ndpa/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=5c410220f5aa036f28bc076499108d3c6668e5fdc6bfc2322e2b5f585e8a9c10
  Stored in directory: /root/.cache/

In [2]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version 20200924  --apt-packages libomp5 libopenblas-dev

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5115  100  5115    0     0  24473      0 --:--:-- --:--:-- --:--:-- 24473
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200924 ...
     |████████████████████████████████| 61kB 2.3MB/s 
Uninstalling torch-1.6.0+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
Done updating TPU runtime
  Successfully uninstalled torch-1.6.0+cu101
Uninstalling torchvision-0.7.0+cu101:
  Successfully uninstalled torchvision-0.7.0+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly+20200924-cp36-cp36m-linux_x86_64.whl...
| [1 files][113.8 MiB/113.8 MiB]                                                
Operation completed over 1 objects/113.8 MiB.                                    


In [3]:
import os
os.environ['XLA_USE_BF16'] = "1"
os.environ['XLA_TENSOR_ALLOCATOR_MAXSIZE'] = '100000000'

import gc
gc.enable()

import numpy as np
import pandas as pd

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader #, Dataset
from torch.utils.data.distributed import DistributedSampler

import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.utils.serialization as xser
import torch_xla.version as xv

import warnings
warnings.filterwarnings("ignore")

In [4]:
import random

def my_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

my_seed()

## Model

In [5]:
WRAPPED_MODEL = xmp.MpModelWrapper(AutoModelForSequenceClassification.from_pretrained("roberta-large"))
tokenizer = AutoTokenizer.from_pretrained("roberta-large")

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

## Data preprocessing

In [6]:
import pandas as pd

In [7]:
train_df = pd.read_csv('train_data.csv')

In [8]:
train_df.head()

,Id,Название,Место,Отдел,Зарплата,Описание компании,Описание вакансии,Требования,Соцпакет,Дистанционно,Вопросы,Тип занятости,Опыт,Образование,Индустрия,Позиция,Фейк
0,10497,"Reporter, Agri Investor","US, ,",KHR,NaN,PEI Media (#URL_4cffe430b8da4c0d7e6074d11e638d...,Seeking savvy business reporter Are you a news...,The successful candidate should have a bachelo...,PEI Media (#URL_4cffe430b8da4c0d7e6074d11e638d...,0,0,Full-time,Associate,Bachelor's Degree,Publishing,Writing/Editing,0
1,8219,Software Engineer,"US, VA, Herndon",Software Engineering,NaN,Roka Security is a boutique security firm that...,Roka Security is looking for Software Engineer...,NaN,NaN,0,0,Full-time,Not Applicable,Unspecified,Computer Software,Engineering,0
2,5342,Structural Engineer,"NL, ZH, Rotterdam",onshore,NaN,NaN,Job tasks:Analyse project specifications on st...,Job requirements:HTS/TU Civil Engineering or M...,NaN,0,1,NaN,NaN,NaN,NaN,NaN,0
3,1916,Senior Publishing Specialist,"US, ,",NaN,NaN,NaN,The Senior Publishing Specialist will be part ...,Minimum of 3 – 4 years experience in Regulator...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0
4,1811,Sales Representative,"CA, BC, Vancouver",Sales & Marketing,NaN,NaN,BCCC Enterprises provides management consultin...,Key Responsibilities:Contribute to the financi...,How to ApplyThis is an exciting opportunity. ...,0,1,NaN,NaN,NaN,NaN,NaN,0


In [9]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10728 entries, 0 to 10727
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Id                 10728 non-null  int64 
 1   Название           10728 non-null  object
 2   Место              10504 non-null  object
 3   Отдел              3758 non-null   object
 4   Зарплата           1730 non-null   object
 5   Описание компании  8682 non-null   object
 6   Описание вакансии  10728 non-null  object
 7   Требования         9097 non-null   object
 8   Соцпакет           6397 non-null   object
 9   Дистанционно       10728 non-null  int64 
 10  Вопросы            10728 non-null  int64 
 11  Тип занятости      8586 non-null   object
 12  Опыт               6478 non-null   object
 13  Образование        5836 non-null   object
 14  Индустрия          7775 non-null   object
 15  Позиция            6853 non-null   object
 16  Фейк               10728 non-null  int64

In [10]:
train_df['Фейк'].value_counts()

0    10205
1      523
Name: Фейк, dtype: int64

In [11]:
train_df['Дистанционно'] = train_df['Дистанционно'].map({1:'Telecommuting positions', 0:'Office positions'})

In [12]:
train_df['Дистанционно'].value_counts()

Office positions           10274
Telecommuting positions      454
Name: Дистанционно, dtype: int64

In [13]:
train_df['Вопросы'] = train_df['Вопросы'].map({1:'Screening questions are present', 0:'There are no screening questions'})

In [14]:
train_df['Вопросы'].value_counts()

There are no screening questions    5424
Screening questions are present     5304
Name: Вопросы, dtype: int64

In [15]:
columns_to_text = ['Название', 
                   'Тип занятости',
                   'Дистанционно',
                #    'Место',
                   'Вопросы', 
                   'Описание компании', 
                   'Описание вакансии', 
                   'Требования']

In [16]:
# columns_to_text = ['Название', 
#                    'Тип занятости',
#                    'Дистанционно',
#                    'Вопросы',
#                 #    'Зарплата',                   
#                 #    'Позиция',
#                    'Образование',
#                    'Опыт',
#                 #    'Место',
#                 #    'Индустрия',
#                 #    'Отдел', 
#                    'Описание компании', 
#                    'Описание вакансии', 
#                    'Требования',
#                    'Соцпакет']

In [17]:
old_columns = train_df.columns

In [18]:
old_columns = old_columns.drop('Фейк')

In [19]:
old_columns

Index(['Id', 'Название', 'Место', 'Отдел', 'Зарплата', 'Описание компании',
       'Описание вакансии', 'Требования', 'Соцпакет', 'Дистанционно',
       'Вопросы', 'Тип занятости', 'Опыт', 'Образование', 'Индустрия',
       'Позиция'],
      dtype='object')

In [20]:
train_df.fillna('', inplace=True)

In [21]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10728 entries, 0 to 10727
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Id                 10728 non-null  int64 
 1   Название           10728 non-null  object
 2   Место              10728 non-null  object
 3   Отдел              10728 non-null  object
 4   Зарплата           10728 non-null  object
 5   Описание компании  10728 non-null  object
 6   Описание вакансии  10728 non-null  object
 7   Требования         10728 non-null  object
 8   Соцпакет           10728 non-null  object
 9   Дистанционно       10728 non-null  object
 10  Вопросы            10728 non-null  object
 11  Тип занятости      10728 non-null  object
 12  Опыт               10728 non-null  object
 13  Образование        10728 non-null  object
 14  Индустрия          10728 non-null  object
 15  Позиция            10728 non-null  object
 16  Фейк               10728 non-null  int64

In [22]:
train_df['text_data'] = train_df[columns_to_text].agg(' '.join, axis=1)

In [23]:
from pprint import pprint

In [24]:
for i in range(10):
    pprint(train_df['text_data'][i])
    print()

('Reporter, Agri Investor Full-time Office positions There are no screening '
 'questions PEI Media\xa0'
 '(#URL_4cffe430b8da4c0d7e6074d11e638d28a1e165c8782b891e111fc17606c3d961#) '
 'provides a range of highly respected publications and market leading events '
 'to its financial sector audience. The\xa0leading information provider in '
 'private capital markets, PEI Media also has a strong international presence, '
 'with offices in London, New York and Hong Kong. Seeking savvy business '
 'reporter Are you a news-hungry journalist? Have a passion for finance, a '
 'talent for breaking news, networking and developing sophisticated sources?If '
 'so, there’s an exciting full-time reporter position available with PEI '
 'Media, the leading global B2B financial publishing and information group '
 'focused exclusively on alternative asset classes.Based in Manhattan, this '
 'opening is a rare opportunity for an ambitious individual seeking to advance '
 'their career in one of the most dy

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
train_df, val_df = train_test_split(train_df, test_size=1024, random_state=42)

In [27]:
val_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1024 entries, 592 to 6311
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Id                 1024 non-null   int64 
 1   Название           1024 non-null   object
 2   Место              1024 non-null   object
 3   Отдел              1024 non-null   object
 4   Зарплата           1024 non-null   object
 5   Описание компании  1024 non-null   object
 6   Описание вакансии  1024 non-null   object
 7   Требования         1024 non-null   object
 8   Соцпакет           1024 non-null   object
 9   Дистанционно       1024 non-null   object
 10  Вопросы            1024 non-null   object
 11  Тип занятости      1024 non-null   object
 12  Опыт               1024 non-null   object
 13  Образование        1024 non-null   object
 14  Индустрия          1024 non-null   object
 15  Позиция            1024 non-null   object
 16  Фейк               1024 non-null   int64

In [28]:
val_df.drop(columns=old_columns, inplace=True)

In [29]:
val_df

,Фейк,text_data
592,0,Technical Site Representative Full-time Offic...
608,0,Communications / Media Relations Full-time Of...
424,0,Customer Support Associate Office positions S...
10133,0,Property Manager (LCAM) Full-time Office posit...
5635,0,Talent Acquisition Specialist USA Full-time Of...
...,...,...
5269,0,Marketing Manager Full-time Office positions T...
7877,0,Healthcare Assistant Office positions There a...
4076,0,Clinical Psychotherapist Full-time Office posi...
106,0,Accountant / Office Manager Full-time Office p...


In [30]:
fake_df = train_df[train_df['Фейк']==1]

In [31]:
fake_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 466 entries, 9413 to 2433
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Id                 466 non-null    int64 
 1   Название           466 non-null    object
 2   Место              466 non-null    object
 3   Отдел              466 non-null    object
 4   Зарплата           466 non-null    object
 5   Описание компании  466 non-null    object
 6   Описание вакансии  466 non-null    object
 7   Требования         466 non-null    object
 8   Соцпакет           466 non-null    object
 9   Дистанционно       466 non-null    object
 10  Вопросы            466 non-null    object
 11  Тип занятости      466 non-null    object
 12  Опыт               466 non-null    object
 13  Образование        466 non-null    object
 14  Индустрия          466 non-null    object
 15  Позиция            466 non-null    object
 16  Фейк               466 non-null    int64

In [32]:
# from random import shuffle

In [33]:
# for i in range(19):
#     shuffle(columns_to_text)
#     print(columns_to_text)
#     fake_df['text_data'] = fake_df[columns_to_text].agg(' '.join, axis=1)
#     train_df = pd.concat([train_df, fake_df], axis=0, ignore_index=True)

In [34]:
train_df = pd.concat([train_df] + [fake_df]*18, axis=0, ignore_index=True)

In [35]:
train_df.shape

(18092, 18)

In [36]:
train_df.drop(columns=old_columns, inplace=True)

In [37]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18092 entries, 0 to 18091
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Фейк       18092 non-null  int64 
 1   text_data  18092 non-null  object
dtypes: int64(1), object(1)
memory usage: 282.8+ KB


In [38]:
train_df['Фейк'].value_counts()

0    9238
1    8854
Name: Фейк, dtype: int64

### Test

In [39]:
test_df = pd.read_csv('test_data.csv')

In [40]:
test_df

,Id,Название,Место,Отдел,Зарплата,Описание компании,Описание вакансии,Требования,Соцпакет,Дистанционно,Вопросы,Тип занятости,Опыт,Образование,Индустрия,Позиция
0,10728,Internet Marketing Specialist,"IE, L, Dublin",Marketing,NaN,NaN,This role represents an important position wit..., 3 to 10+ years experience in a similar inter...,RemunerationA highly attractive salary will be...,0,0,NaN,NaN,NaN,NaN,NaN
1,10729,Business Development,"DE, , Berlin",NaN,NaN,We are on a mission: Making cities smarter.Cit...,You have an opinion about how we can make our ...,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
2,10730,Sonographer - Full and Part Time Opportunities,"GB, ,",Ultrasound,NaN,Diagnostic Health deploy and manage high quali...,We are a leading local ultrasound company spec...,Diploma of College of Radiographers or BSc (Ho...,Competative SalaryHours to suit,0,1,Part-time,NaN,Associate Degree,Medical Practice,Health Care Provider
3,10731,Care Support Worker,"GB, EDH, Edinburgh",NaN,NaN,Social Care Alba is the highest graded Care at...,Social Care Alba is the highest quality Care a...,Key Accountabilities would be:To provide indiv...,This is your chance to become part of our cont...,0,1,NaN,Not Applicable,NaN,Hospital & Health Care,NaN
4,10732,Outside Sales Professional-Springfield,"US, IL, Springfield",NaN,NaN,"ABC Supply Co., Inc. is the nation’s largest w...","As an Outside Sales Representative, you must h...",Track Record of Sales Success – B2B or B2CNo m...,"As an Outside Sales Representative, you will r...",0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7147,17875,System Engineer (jr),NaN,NaN,NaN,Massive Media is the social media company behi...,FunctionThe IT Services-department serves our ...,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
7148,17876,Mirfield Warehousing Apprenticeship Under NAS ...,"GB, , Mirfield",NaN,NaN,Established on the principles that full time e...,Government funding is only available for 16-1...,16-18 year olds only due to government funding...,Career prospects.,0,1,Full-time,Not Applicable,High School or equivalent,Warehousing,NaN
7149,17877,Sales Director,"US, PA, Philadelphia",NaN,NaN,Visual BI is one of the leading and fastest gr...,City / Location : Any Hub City in the USA Join...,"New Business Development• Prospect, qualify an...",Salary and bonus commensurate with experience....,1,1,Full-time,Director,Associate Degree,Information Technology and Services,Sales
7150,17878,Sales Development Representative,"US, CA, Emeryville",Sales,NaN,Abakus is a software platform built on game th...,Abakus is a software platform built on game th...,&lt;1 year prior Sales Development experience ...,Competitive salaryGenerous bonus/commission ba...,0,0,Full-time,Entry level,Bachelor's Degree,Computer Software,Sales


In [41]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7152 entries, 0 to 7151
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Id                 7152 non-null   int64 
 1   Название           7152 non-null   object
 2   Место              7030 non-null   object
 3   Отдел              2575 non-null   object
 4   Зарплата           1138 non-null   object
 5   Описание компании  5890 non-null   object
 6   Описание вакансии  7151 non-null   object
 7   Требования         6088 non-null   object
 8   Соцпакет           4273 non-null   object
 9   Дистанционно       7152 non-null   int64 
 10  Вопросы            7152 non-null   int64 
 11  Тип занятости      5823 non-null   object
 12  Опыт               4352 non-null   object
 13  Образование        3939 non-null   object
 14  Индустрия          5202 non-null   object
 15  Позиция            4572 non-null   object
dtypes: int64(3), object(13)
memory usage: 894.

In [42]:
test_df['Дистанционно'] = test_df['Дистанционно'].map({1:'Telecommuting positions', 0:'Office positions'})

In [43]:
test_df['Вопросы'] = test_df['Вопросы'].map({1:'Screening questions are present', 0:'There are no screening questions'})

In [44]:
# columns_to_text = ['Название', 
#                    'Тип занятости',
#                    'Дистанционно',
#                    'Место',
#                    'Вопросы', 
#                    'Описание компании', 
#                    'Описание вакансии', 
#                    'Требования']

In [45]:
test_df.fillna('', inplace=True)

In [46]:
test_df['text_data'] = test_df[columns_to_text].agg(' '.join, axis=1)

In [47]:
test_df.drop(columns=old_columns, inplace=True)

In [48]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7152 entries, 0 to 7151
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text_data  7152 non-null   object
dtypes: object(1)
memory usage: 56.0+ KB


## Datasets

In [49]:
import nlp

In [50]:
MAX_LEN = 512 #256

def convert_to_features(batch):

    encodings = tokenizer.batch_encode_plus(batch['text_data'], 
                                            add_special_tokens=True,
                                            pad_to_max_length=True,
                                            truncation=True,
                                            max_length=MAX_LEN)
    return encodings

train_dataset = nlp.Dataset.from_pandas(train_df) #nlp.load_dataset('csv', data_files='train_data.csv') #
# columns_to_drop = train_dataset.column_names #train_dataset['train'].column_names
# columns_to_drop.remove('Название')
# columns_to_drop.remove('Описание вакансии')
# columns_to_drop.remove('Фейк')
# train_dataset = train_dataset.map(lambda example: {'Описание вакансии': example['Название'] + ' ' + example['Тип занятости'] + ' ' + example['Описание вакансии'] + ' ' + example['Описание компании'] + ' ' + example['Требования']})
train_dataset = train_dataset.map(convert_to_features, batched=True) #, remove_columns=columns_to_drop)
# train_dataset = train_dataset['train'].train_test_split(test_size=0.1, shuffle=True, seed=42)
train_dataset.set_format("torch", columns=['input_ids', 'attention_mask', 'Фейк'])

valid_dataset = nlp.Dataset.from_pandas(val_df)
# valid_dataset = valid_dataset.map(lambda example: {'Описание вакансии': example['Название'] + ' ' + example['Тип занятости'] + ' ' + example['Описание вакансии'] + ' ' + example['Описание компании'] + ' ' + example['Требования']})
valid_dataset = valid_dataset.map(convert_to_features, batched=True) #, remove_columns=columns_to_drop)
# valid_dataset = valid_dataset.map(convert_to_features, batched=True) 
valid_dataset.set_format("torch", columns=['input_ids', 'attention_mask', 'Фейк'])

test_dataset = nlp.Dataset.from_pandas(test_df) #nlp.load_dataset('csv', data_files={'test':'test_data.csv'}) #
# test_dataset = test_dataset.map(lambda example: {'Описание вакансии': example['Название'] + ' ' + example['Тип занятости'] + ' ' + example['Описание вакансии'] + ' ' + example['Описание компании'] + ' ' + example['Требования']})
test_dataset = test_dataset.map(convert_to_features, batched=True) #, remove_columns=columns_to_drop) 
test_dataset.set_format("torch", columns=['input_ids', 'attention_mask'])

In [51]:
# test_df = pd.read_csv('test_data.csv')
# test_df.info()

In [52]:
# columns_to_drop.remove('Название')
# test_df.drop(columns=columns_to_drop, inplace=True)
# test_df.fillna('', inplace=True)
# test_df.info()

In [53]:
# test_dataset = nlp.Dataset.from_pandas(test_df)
# test_dataset = test_dataset.map(lambda example: {'Описание вакансии': example['Название'] + ' ' + example['Описание вакансии']})
# test_dataset = test_dataset.map(convert_to_features, batched=True) 
# test_dataset.set_format("torch", columns=['input_ids', 'attention_mask'])

## Train & Submit

### Metrics

In [54]:
from sklearn.metrics import f1_score

In [55]:
class AverageMeter(object):
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {avg' + self.fmt + '}'
        return fmtstr.format(**self.__dict__)

class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

def F1_score(outputs, targets):
    with torch.no_grad():
        return f1_score(targets, outputs.argmax(dim=-1)) #, average='micro') 

### Training

In [56]:
def train_loop_fn(train_loader, model, optimizer, device, epoch=None):

    model.train()
    
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter(' F1', ':.2f')
    progress = ProgressMeter(
        len(train_loader),
        [losses, top1],
        prefix="[xla:{}]  Train:  Epoch: [{}]".format(xm.get_ordinal(), epoch + 1)
    )    

    for i, data in enumerate(train_loader):

        input_ids = data["input_ids"].to(device, dtype=torch.long)
        attention_mask = data["attention_mask"].to(device, dtype=torch.long)
        labels = data["Фейк"].to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(
            input_ids = input_ids,
            attention_mask = attention_mask,
            labels = labels
        )

        loss = outputs[0] 
        loss.backward()
        xm.optimizer_step(optimizer)

        f1 = F1_score(outputs[1].cpu(), labels.cpu())
        losses.update(loss.item(), input_ids.size(0))
        top1.update(f1.item(), input_ids.size(0))

        if i == len(train_loader) - 1: 
            progress.display(i)

    del loss
    del f1
    del outputs
    del input_ids
    del attention_mask
    del labels
    gc.collect()

### Evaluation

In [57]:
from sklearn.metrics import confusion_matrix

In [58]:
def eval_loop_fn(test_data_loader, model, device):

    model.eval()

    with torch.no_grad():

        test_preds = []
        targets = []
        val_loss = 0

        for i, data in enumerate(test_data_loader):

            input_ids = data["input_ids"].to(device, dtype=torch.long)
            attention_mask = data["attention_mask"].to(device, dtype=torch.long)
            labels = data["Фейк"].to(device, dtype=torch.float)

            outputs = model(
                input_ids = input_ids,
                attention_mask = attention_mask,
                labels = labels
            )

            val_loss += outputs[0].item()
            test_preds.extend(torch.argmax(outputs[1], -1).cpu().detach().tolist())
            targets.extend(data["Фейк"].detach().tolist())  
        
    xm.master_print(f'Val loss: {val_loss / len(test_data_loader)}')
    xm.master_print(f'F1 score: {f1_score(targets, test_preds)}')
    xm.master_print('Confusion matrix:')
    xm.master_print(f'{confusion_matrix(targets, test_preds)}')

    del outputs
    del targets
    del val_loss
    del test_preds
    del input_ids
    del attention_mask
    gc.collect()

### Test

In [59]:
sample_submission = pd.read_csv('sample_submission.csv')

In [60]:
def test_loop_fn(test_data_loader, model, device, epoch=0):

    model.eval()

    with torch.no_grad():

        test_preds = []

        for i, data in enumerate(test_data_loader):

            input_ids = data["input_ids"].to(device, dtype=torch.long)
            attention_mask = data["attention_mask"].to(device, dtype=torch.long)

            outputs = model(
                input_ids = input_ids,
                attention_mask = attention_mask,
            )[0]
            
            outputs_np = outputs.cpu().detach().numpy().tolist()
            test_preds.extend(outputs_np)  
        
    test_preds = torch.FloatTensor(test_preds)

    sample_submission['Фейк'] = torch.argmax(test_preds, -1).cpu().detach().numpy()
    sample_submission.to_csv(f'submission_{epoch + 1}.csv', index=False)

    del outputs
    del outputs_np
    del test_preds
    del input_ids
    del attention_mask
    gc.collect()

### Run

In [61]:
my_seed(42)

TRAIN_BATCH_SIZE = 4 * xm.xrt_world_size()
VALID_BATCH_SIZE = 8 #* xm.xrt_world_size()
TEST_BATCH_SIZE = 8
EPOCHS = 10
LR = 5e-5 * xm.xrt_world_size() 

In [62]:
# from google.colab import drive
# drive.mount('/content/drive')

In [63]:
def _run():

    # Train
    xm.master_print('Starting Run ...')
    train_sampler = DistributedSampler( #DistributedWeightedSampler( #
        train_dataset, #train_dataset['train'],
        num_replicas=xm.xrt_world_size(),
        rank=xm.get_ordinal(),
        shuffle=True 
    )    
    train_data_loader = DataLoader(
        train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        sampler=train_sampler,
        drop_last=True,
        num_workers=0,
    )
    xm.master_print('Train Loader Created.')
    
    # Valid
    # valid_sampler = DistributedSampler(
    #     train_dataset['test'],
    #     num_replicas=xm.xrt_world_size(),
    #     rank=xm.get_ordinal(),
    #     shuffle=False
    # )    
    valid_data_loader = DataLoader(
        valid_dataset,
        batch_size=VALID_BATCH_SIZE,
        # sampler=valid_sampler,
        drop_last=False,
        num_workers=0,
        shuffle=False,
    )
    xm.master_print('Valid Loader Created.')

    # Test
    test_data_loader = DataLoader(
        test_dataset, 
        batch_size=TEST_BATCH_SIZE,
        drop_last=False,
        num_workers=0,
        shuffle=False,
    )
    xm.master_print('Test Loader Created.')
    
    num_train_steps = int(train_dataset.num_rows / TRAIN_BATCH_SIZE / xm.xrt_world_size())
    device = xm.xla_device()
    model = WRAPPED_MODEL.to(device)
    xm.master_print('Done Model Loading.')
    optimizer = AdamW(
        model.parameters(),
        lr=LR, 
    )
    xm.master_print(f'Num Train Steps= {num_train_steps}, XRT World Size= {xm.xrt_world_size()}.')

    # para_loader = pl.ParallelLoader(valid_data_loader, [device])
    # xm.master_print('Parallel Loader Created. Validating ...')
    # eval_loop_fn(para_loader.per_device_loader(device), 
    #                  model,  
    #                  device
    #                 )
    
    for epoch in range(EPOCHS):

        # !
        train_sampler.set_epoch(epoch)

        para_loader = pl.ParallelLoader(train_data_loader, [device])
        xm.master_print('Parallel Loader Created. Training ...')
        train_loop_fn(para_loader.per_device_loader(device),
                      model,  
                      optimizer, 
                      device,  
                      epoch
                     )        
        xm.master_print("Finished training epoch {}".format(epoch + 1))
            
        para_loader = pl.ParallelLoader(valid_data_loader, [device])
        xm.master_print('Parallel Loader Created. Validating ...')
        eval_loop_fn(para_loader.per_device_loader(device), 
                     model,  
                     device
                    )
        
        para_loader = pl.ParallelLoader(test_data_loader, [device])
        xm.master_print('Parallel Loader Created. Testing ...')
        test_loop_fn(para_loader.per_device_loader(device), 
                    model,  
                    device,
                    epoch 
                    )
        xm.master_print('Predicted.')

        # if epoch == EPOCHS-1:
        #     xm.save(model.state_dict(), f'/content/drive/My Drive/RoBERTa_{epoch + 1}.pt')
        #     xm.master_print('Model Saved.')

In [64]:
def _mp_fn(rank, flags):
    _run()

FLAGS={}
xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method='fork')

Starting Run ...
Train Loader Created.
Valid Loader Created.
Test Loader Created.
Done Model Loading.
Num Train Steps= 565, XRT World Size= 8.
Parallel Loader Created. Training ...
[xla:2]  Train:  Epoch: [1][564/565]	Loss 1.1932e-01	 F1 0.88
[xla:1]  Train:  Epoch: [1][564/565]	Loss 1.1027e-01	 F1 0.87
[xla:4]  Train:  Epoch: [1][564/565]	Loss 1.1992e-01	 F1 0.89
[xla:3]  Train:  Epoch: [1][564/565]	Loss 1.2601e-01	 F1 0.87
[xla:0]  Train:  Epoch: [1][564/565]	Loss 1.2175e-01	 F1 0.87
[xla:7]  Train:  Epoch: [1][564/565]	Loss 1.2366e-01	 F1 0.87
[xla:6]  Train:  Epoch: [1][564/565]	Loss 1.2880e-01	 F1 0.86
[xla:5]  Train:  Epoch: [1][564/565]	Loss 1.1696e-01	 F1 0.86
Finished training epoch 1
Parallel Loader Created. Validating ...
Val loss: 0.043856799602508545
F1 score: 0.8807339449541285
Confusion matrix:
[[963   4]
 [  9  48]]
Parallel Loader Created. Testing ...
Predicted.
Parallel Loader Created. Training ...
[xla:7]  Train:  Epoch: [2][564/565]	Loss 7.2519e-03	 F1 0.93
[xla:4] 